# 5. Exploring genetic algorithms

Let's explore existing genetic algorithms implementations in Python, and then use them in a new third model.

## Search

[This Google search was conducted](https://www.google.com/search?q=python+genetic+algorithm&sca_esv=576325651&ei=e3E4ZfXZGLzN1sQPnv-KyAg&ved=0ahUKEwi1g-WMiJCCAxW8ppUCHZ6_AokQ4dUDCBA&uact=5&oq=python+genetic+algorithm&gs_lp=Egxnd3Mtd2l6LXNlcnAiGHB5dGhvbiBnZW5ldGljIGFsZ29yaXRobTIFEAAYgAQyBRAAGIAEMgYQABgHGB4yBhAAGAcYHjIGEAAYBxgeMgYQABgHGB4yBhAAGAcYHjIEEAAYHjIEEAAYHjIEEAAYHkjkEVCAC1iMDnABeAGQAQCYAYgBoAHlBaoBAzAuNrgBA8gBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on 2023-10-24. The following libraries were found:

* [PyGAD](https://pygad.readthedocs.io/en/latest/)
* [geneticalgorithm](https://pygad.readthedocs.io/en/latest/)

**The first one seems to be more complete**. Let's try it.

## Exploring